BONOS - VPN - DURACIÓN - DURACIÓN MODOFICADA - CONVEXIDAD

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import datetime as dt

from datetime import date
from dateutil.relativedelta import relativedelta
from datetime import datetime
#from datetime import timedelta

In [ ]:
# VALOR PRESENTE NETO = VPN
# PRECIO SUCIO = VPN
# PRECIO LIMPIO = PRECIO SUCIO - CUPON CORRIDO
# CUPON CORRIDO = (CUPON / BASE )* DIAS CORRIDOS


"""
Calculadora de Precios de Bonos (Sucio, Limpio, Cupón Corrido)
"""


# --- PARÁMETROS DE ENTRADA ---

# Fechas
fecha_liquidacion = datetime.date(2024, 7, 15) # Fecha en que se compra/valora el bono
fecha_emision = datetime.date(2023, 1, 1)
fecha_vencimiento = datetime.date(2028, 1, 1)

# Características del Bono
valor_nominal = 1000.0       # Valor facial del bono
tasa_cupon_anual = 0.06    # Tasa de interés anual del cupón (ej. 6%)
frecuencia_pago_cupon = 2  # Pagos por año (1: Anual, 2: Semestral, 4: Trimestral)

# Tasa de Mercado / Descuento
tasa_descuento_anual = 0.05 # Tasa de rendimiento requerida o YTM (ej. 5%)

# --- CÁLCULOS INTERMEDIOS ---

# 1. Calcular Fechas de Pago de Cupones
intervalo_meses = 12 // frecuencia_pago_cupon
fechas_cupones = []
fecha_temp = fecha_emision + relativedelta(months=intervalo_meses)
while fecha_temp <= fecha_vencimiento:
    fechas_cupones.append(fecha_temp)
    fecha_temp += relativedelta(months=intervalo_meses)

# Asegurarse que la fecha de vencimiento esté (podría no coincidir exactamente si hay días extra)
# La última fecha debe ser la de vencimiento para el pago del principal.
if not fechas_cupones or fechas_cupones[-1] != fecha_vencimiento:
     # Podríamos ajustar la última fecha o añadirla si falta
     # Por simplicidad, asumimos que el último cupón coincide con el vencimiento
     if fechas_cupones and fechas_cupones[-1] > fecha_vencimiento:
         fechas_cupones[-1] = fecha_vencimiento # Ajustar si se pasó
     elif not fechas_cupones or fechas_cupones[-1] < fecha_vencimiento:
          # Si la última fecha calculada es menor, puede que necesitemos añadir vencimiento
          # Solo añadir si es diferente a la última fecha para evitar duplicados
          if not fechas_cupones or fecha_vencimiento != fechas_cupones[-1]:
              fechas_cupones.append(fecha_vencimiento) # Añadir si falta


# 2. Calcular Flujos de Caja Futuros
monto_cupon_periodico = (tasa_cupon_anual * valor_nominal) / frecuencia_pago_cupon
tasa_descuento_periodica = tasa_descuento_anual / frecuencia_pago_cupon

flujos_futuros = []
fechas_flujos_futuros = []

for fecha_cupon in fechas_cupones:
    if fecha_cupon > fecha_liquidacion:
        fechas_flujos_futuros.append(fecha_cupon)
        if fecha_cupon == fecha_vencimiento:
            flujos_futuros.append(monto_cupon_periodico + valor_nominal) # Cupón + Principal
        else:
            flujos_futuros.append(monto_cupon_periodico) # Solo Cupón

# 3. Encontrar Fechas de Cupón Relevantes para Cupón Corrido
fecha_ultimo_cupon = fecha_emision # Empezar desde la emisión
fecha_proximo_cupon = None

# Encontrar el último cupón PAGADO ANTES o EN la fecha de liquidación
fechas_cupones_pagados = [fc for fc in fechas_cupones if fc <= fecha_liquidacion]
if fechas_cupones_pagados:
    fecha_ultimo_cupon = max(fechas_cupones_pagados)
# Si no hay cupones pasados, el último "cupón" fue en la fecha de emisión
elif fecha_liquidacion >= fecha_emision:
     fecha_ultimo_cupon = fecha_emision
else: # Liquidación antes de emisión (no tiene sentido)
     fecha_ultimo_cupon = None


# Encontrar el primer cupón A PAGAR DESPUÉS de la fecha de liquidación
fechas_cupones_siguientes = [fc for fc in fechas_cupones if fc > fecha_liquidacion]
if fechas_cupones_siguientes:
    fecha_proximo_cupon = min(fechas_cupones_siguientes)
else:
    # Si no hay cupones siguientes, estamos en o después del vencimiento
    if fecha_liquidacion >= fecha_vencimiento:
         fecha_proximo_cupon = fecha_vencimiento # O None si se prefiere indicar que no hay próximo
    else:
         # Esto podría indicar un error en la generación de fechas de cupón
         print("Advertencia: No se encontró un próximo cupón aunque la liquidación es antes del vencimiento.")
         # Intentar calcularlo basado en el último + intervalo como fallback
         if fecha_ultimo_cupon and fecha_ultimo_cupon != fecha_vencimiento:
              fecha_proximo_cupon_calc = fecha_ultimo_cupon + relativedelta(months=intervalo_meses)
              fecha_proximo_cupon = min(fecha_proximo_cupon_calc, fecha_vencimiento)
         else:
             fecha_proximo_cupon = fecha_vencimiento


# --- CÁLCULO DEL CUPÓN CORRIDO ---

cupon_corrido = 0.0
dias_corridos = 0
dias_periodo_cupon = 1 # Evitar división por cero

# Solo hay cupón corrido si la liquidación es ESTRICTAMENTE después del último cupón

if fecha_ultimo_cupon and fecha_proximo_cupon and \
   fecha_liquidacion > fecha_ultimo_cupon and \
   fecha_liquidacion < fecha_proximo_cupon and \
   fecha_ultimo_cupon != fecha_proximo_cupon:

    dias_corridos = (fecha_liquidacion - fecha_ultimo_cupon).days
    dias_periodo_cupon = (fecha_proximo_cupon - fecha_ultimo_cupon).days

    if dias_periodo_cupon > 0:
      cupon_corrido = monto_cupon_periodico * (dias_corridos / dias_periodo_cupon)

# Si la liquidación es en una fecha de cupón, el cupón corrido es cero porque se acaba de pagar/recibir.
elif fecha_liquidacion in fechas_cupones:
    cupon_corrido = 0.0
    dias_corridos = 0


# --- CÁLCULO DEL VALOR PRESENTE NETO (VPN) / PRECIO SUCIO ---
# El VPN de los flujos de caja futuros descontados a la fecha de liquidación.

precio_sucio_vpn = 0.0

if fecha_liquidacion < fecha_vencimiento and fecha_proximo_cupon: # Solo calcular si hay flujos futuros
    # Calcular días desde liquidación hasta el próximo cupón
    dias_hasta_proximo = (fecha_proximo_cupon - fecha_liquidacion).days
    # Calcular días en el período de cupón que contiene la fecha de liquidación
    dias_periodo_actual = 1 # Default
    if fecha_ultimo_cupon and fecha_proximo_cupon and fecha_ultimo_cupon != fecha_proximo_cupon:
         dias_periodo_actual = (fecha_proximo_cupon - fecha_ultimo_cupon).days

    if dias_periodo_actual <= 0: dias_periodo_actual = 1 # Evitar división por cero o negativo

    # Fracción del período desde la liquidación hasta el próximo cupón (w en la fórmula)
    # Si liquidación es fecha de cupón, w = 0 (o días_hasta_proximo = 0)
    fraccion_periodo_restante = dias_hasta_proximo / dias_periodo_actual if dias_periodo_actual > 0 else 0

    # Asegurarse que w esté entre 0 y 1 (puede dar negativo si las fechas son inconsistentes)
    fraccion_periodo_restante = max(0, min(1, fraccion_periodo_restante))

    valor_presente_total = 0
    for i in range(len(flujos_futuros)):
        flujo = flujos_futuros[i]
        # El exponente es el número de periodos completos MÁS la fracción inicial
        # i=0 es el primer flujo futuro (que ocurre después de 'w' fracción de periodo)
        # Exponente = i + w
        exponente_descuento = i + fraccion_periodo_restante
        valor_presente_total += flujo / ((1 + tasa_descuento_periodica) ** exponente_descuento)

    precio_sucio_vpn = valor_presente_total

elif fecha_liquidacion == fecha_vencimiento:
     # Si liquidamos exactamente en vencimiento, el precio es el valor nominal más el último cupón (si aplica).
     # No hay descuento. El cupón corrido es 0.
     print("Nota: Liquidación en fecha de vencimiento.")
     # Asumimos que el último cupón se paga en esta fecha.
     precio_sucio_vpn = valor_nominal + monto_cupon_periodico
     cupon_corrido = 0.0 # No hay interés acumulado si se liquida en la fecha de pago
elif fecha_liquidacion > fecha_vencimiento:
     print("Advertencia: La fecha de liquidación es posterior al vencimiento. El valor es 0.")
     precio_sucio_vpn = 0.0
     cupon_corrido = 0.0

# --- CÁLCULO DEL PRECIO LIMPIO ---
precio_limpio = precio_sucio_vpn - cupon_corrido

# --- RESULTADOS ---
print("--- Cálculo de Precios de Bono ---")
print(f"Fecha de Liquidación: {fecha_liquidacion}")
print(f"Fecha de Vencimiento: {fecha_vencimiento}")
print(f"Valor Nominal: {valor_nominal:.2f}")
print(f"Tasa Cupón Anual: {tasa_cupon_anual:.2%}")
print(f"Frecuencia de Pago: {frecuencia_pago_cupon} veces/año")
print(f"Tasa Descuento Anual (YTM): {tasa_descuento_anual:.2%}")
print("-" * 30)
print(f"Fecha Último Cupón (antes o en liquidación): {fecha_ultimo_cupon}")
print(f"Fecha Próximo Cupón (después de liquidación): {fecha_proximo_cupon}")
print(f"Monto Cupón Periódico: {monto_cupon_periodico:.2f}")
print(f"Días Corridos desde último cupón (excl. liquidación): {dias_corridos}")
print(f"Días en el Periodo de Cupón Actual: {dias_periodo_actual if fecha_ultimo_cupon and fecha_proximo_cupon else 'N/A'}")
print("-" * 30)
print(f"Cupón Corrido (Intereses Devengados): {cupon_corrido:.6f}")
print(f"Precio Sucio (VPN de flujos futuros): {precio_sucio_vpn:.6f}")
print(f"Precio Limpio (Precio Sucio - Cupón Corrido): {precio_limpio:.6f}")

# Nota sobre precios: A menudo se expresan como % del valor nominal
print("-" * 30)
print(f"Precio Sucio (% del Nominal): {precio_sucio_vpn / valor_nominal * 100:.6f}%")
print(f"Precio Limpio (% del Nominal): {precio_limpio / valor_nominal * 100:.6f}%")

--- Cálculo de Precios de Bono ---
Fecha de Liquidación: 2024-07-15
Fecha de Vencimiento: 2028-01-01
Valor Nominal: 1000.00
Tasa Cupón Anual: 6.00%
Frecuencia de Pago: 2 veces/año
Tasa Descuento Anual (YTM): 5.00%
------------------------------
Fecha Último Cupón (antes o en liquidación): 2024-07-01
Fecha Próximo Cupón (después de liquidación): 2025-01-01
Monto Cupón Periódico: 30.00
Días Corridos desde último cupón (excl. liquidación): 14
Días en el Periodo de Cupón Actual: 184
------------------------------
Cupón Corrido (Intereses Devengados): 2.282609
Precio Sucio (VPN de flujos futuros): 1033.687207
Precio Limpio (Precio Sucio - Cupón Corrido): 1031.404598
------------------------------
Precio Sucio (% del Nominal): 103.368721%
Precio Limpio (% del Nominal): 103.140460%


In [ ]:
# PRECIO SUCIO

# -*- coding: utf-8 -*-
"""
Cálculo Independiente del Precio Sucio (VPN) de un Bono
"""

import datetime
from dateutil.relativedelta import relativedelta

# --- PARÁMETROS DE ENTRADA ---
# Puedes modificar estos valores según el bono que quieras analizar
fecha_liquidacion = datetime.date(2024, 7, 15)
fecha_emision = datetime.date(2023, 1, 1)
fecha_vencimiento = datetime.date(2028, 1, 1)
valor_nominal = 1000.0
tasa_cupon_anual = 0.06    # Ej: 6% = 0.06
frecuencia_pago_cupon = 2  # 1: Anual, 2: Semestral, 4: Trimestral
tasa_descuento_anual = 0.05 # YTM o tasa de rendimiento requerida. Ej: 5% = 0.05

# --- FUNCIÓN PARA CALCULAR PRECIO SUCIO ---

def calcular_precio_sucio(fecha_liq, fecha_emis, fecha_venc, val_nom, tasa_cup_an, freq_pago, tasa_desc_an):


    if fecha_liq > fecha_venc:
        print("Advertencia: Fecha de liquidación es posterior al vencimiento.")
        return 0.0

    # 1. Calcular Fechas de Pago de Cupones
    intervalo_meses = 12 // freq_pago
    fechas_cupones = []
    # Empezar desde la primera fecha de cupón después de la emisión
    fecha_temp = fecha_emis + relativedelta(months=intervalo_meses)
    while fecha_temp <= fecha_venc:
        fechas_cupones.append(fecha_temp)
        fecha_temp += relativedelta(months=intervalo_meses)
    # Asegurar que la fecha de vencimiento esté incluida (puede ser el último pago)
    if not fechas_cupones or fechas_cupones[-1] < fecha_venc:
        if not fechas_cupones or fecha_venc != fechas_cupones[-1]:
            fechas_cupones.append(fecha_venc)
    elif fechas_cupones[-1] > fecha_venc:
        fechas_cupones[-1] = fecha_venc

    # 2. Calcular Tasas y Montos Periódicos
    monto_cupon_periodico = (tasa_cup_an * val_nom) / freq_pago
    tasa_descuento_periodica = tasa_desc_an / freq_pago

    # 3. Identificar Flujos de Caja Futuros (posteriores a la liquidación)
    flujos_futuros = []
    fechas_flujos_futuros = []
    for fecha_cupon in fechas_cupones:
        if fecha_cupon > fecha_liq:
            fechas_flujos_futuros.append(fecha_cupon)
            if fecha_cupon == fecha_venc:
                flujos_futuros.append(monto_cupon_periodico + val_nom)
            else:
                flujos_futuros.append(monto_cupon_periodico)

    # 4. Encontrar Fechas Relevantes para Descuento Fraccional
    fecha_ultimo_cupon = fecha_emis
    fechas_cupones_pagados = [fc for fc in fechas_cupones if fc <= fecha_liq]
    if fechas_cupones_pagados:
        fecha_ultimo_cupon = max(fechas_cupones_pagados)
    elif fecha_liq < fecha_emis: # Liquidación antes de emisión, no debería pasar
         return 0.0 # O manejar como error

    fecha_proximo_cupon = None
    if fechas_flujos_futuros: # El próximo cupón es la fecha del primer flujo futuro
        fecha_proximo_cupon = fechas_flujos_futuros[0]
    elif fecha_liq == fecha_venc: # Si liquidamos en vencimiento, no hay próximo cupón estricto
         fecha_proximo_cupon = fecha_venc
    # Podría faltar un fallback si no hay flujos futuros pero liq < venc


    # 5. Calcular Fracción de Periodo Restante (w)
    fraccion_periodo_restante = 0.0
    if fecha_liq < fecha_venc and fecha_proximo_cupon and fecha_ultimo_cupon and fecha_proximo_cupon > fecha_ultimo_cupon:
        # Días desde liquidación hasta el próximo cupón
        dias_hasta_proximo = max(0, (fecha_proximo_cupon - fecha_liq).days)
        # Días totales en el período actual (entre último y próximo)
        dias_periodo_actual = (fecha_proximo_cupon - fecha_ultimo_cupon).days

        if dias_periodo_actual > 0:
            fraccion_periodo_restante = dias_hasta_proximo / dias_periodo_actual
            fraccion_periodo_restante = max(0, min(1, fraccion_periodo_restante))
        # else: Si dias_periodo_actual es 0, w permanece 0. Ocurre si liq = ultimo = proximo? Imposible.

    # 6. Calcular Precio Sucio (VPN)
    precio_sucio_calculado = 0.0
    if fecha_liq < fecha_venc:
        if flujos_futuros:
            valor_presente_total = 0.0
            for i in range(len(flujos_futuros)):
                flujo = flujos_futuros[i]
                # Exponente = índice del periodo futuro (empezando en 0) + fracción restante
                exponente_descuento = i + fraccion_periodo_restante
                valor_presente_total += flujo / ((1 + tasa_descuento_periodica) ** exponente_descuento)
            precio_sucio_calculado = valor_presente_total
    elif fecha_liq == fecha_venc:
        # En vencimiento, el valor es el pago final (nominal + cupón) sin descuento
        precio_sucio_calculado = val_nom + monto_cupon_periodico

    return precio_sucio_calculado

# --- EJECUCIÓN DEL CÁLCULO ---

# Llama a la función con los parámetros definidos arriba
precio_sucio = calcular_precio_sucio(
    fecha_liquidacion,
    fecha_emision,
    fecha_vencimiento,
    valor_nominal,
    tasa_cupon_anual,
    frecuencia_pago_cupon,
    tasa_descuento_anual
)

# --- RESULTADO ---
if precio_sucio is not None:
    print(f"--- Cálculo del Precio Sucio ---")
    print(f"Fecha de Liquidación: {fecha_liquidacion}")
    print(f"Fecha de Vencimiento: {fecha_vencimiento}")
    print(f"Valor Nominal: {valor_nominal:.2f}")
    print(f"Tasa Cupón Anual: {tasa_cupon_anual:.2%}")
    print(f"Frecuencia de Pago: {frecuencia_pago_cupon} veces/año")
    print(f"Tasa Descuento Anual (YTM): {tasa_descuento_anual:.2%}")
    print("-" * 30)
    print(f"El Precio Sucio (VPN) calculado es: {precio_sucio:.6f}")
    print(f"Como % del Nominal: {(precio_sucio / valor_nominal * 100) if valor_nominal else 0:.6f}%")

--- Cálculo del Precio Sucio ---
Fecha de Liquidación: 2024-07-15
Fecha de Vencimiento: 2028-01-01
Valor Nominal: 1000.00
Tasa Cupón Anual: 6.00%
Frecuencia de Pago: 2 veces/año
Tasa Descuento Anual (YTM): 5.00%
------------------------------
El Precio Sucio (VPN) calculado es: 1033.687207
Como % del Nominal: 103.368721%


In [ ]:
TASAM = 0.0956
TASAC = 0.10
PERIODOS = 20
NOMINAL = 100
#PAGOS = ANUAL

FLUJOS = []
for i in range(1, PERIODOS):
    CUP=(NOMINAL * TASAC)
    FLUJOS.append(CUP)
    i + 1
    if i == PERIODOS - 1:
      CUP = (NOMINAL * TASAC) + 100
      FLUJOS.append(CUP)


PSUCIO = sum(cf / (1 + TASAM) ** i for i, cf in enumerate(FLUJOS, 1))
PSUCIO



103.8612782000864

In [ ]:
# TENIENDO ENCUENTA FECHAS
fhoy = datetime.today()                 # fecha hoy

fvto = datetime(2032,11,15)
final = fvto - relativedelta(years=1)   # restando 1 año a la fecha final

fvto2 = datetime(2032,11,15)
final2 = fvto - relativedelta(months=+6) # reatando 6 meses  a la fecha final


fechas = []
dias1 = []
años = []
vpn = []
dur = []
durm = []


while fvto > fhoy:

  dias = (fvto - fhoy).days
  años = dias/365
  fechas.append(fvto)
  dias1.append(dias)

  #Calculo del VPN
  vpn1 = FLUJOS[i] / ((1 + TASAM) ** años)
  vpn.append(vpn1)

  #calculo de duracion
  delta_dias = (fvto - fhoy).days
  duracion = delta_dias / 365
  dur.append(duracion)

  #calculo de duración modificada
  durmod = duracion / (1 + TASAM)
  durm.append(durmod)

  #restando un año a la fecha final
  fvto = fvto - relativedelta(years=1)



fechas = pd.DataFrame(fechas)
dias1 = pd.DataFrame(dias1)
vpn = pd.DataFrame(vpn)
dur = pd.DataFrame(dur)
durm = pd.DataFrame(durm)



tabla = pd.concat([fechas, dias1, vpn, dur, durm], axis=1)
tabla.columns = ['Fecha', 'Días', 'VPN', 'Duración', 'Dur-modificada']
tabla



,Fecha,Días,VPN,Duración,Dur-modificada
0,2032-11-15,2775,54.945062,7.60274,6.939339
1,2031-11-15,2409,60.212870,6.60000,6.024096
2,2030-11-15,2044,65.969221,5.60000,5.111355
3,2029-11-15,1679,72.275878,4.60000,4.198613
4,2028-11-15,1314,79.185452,3.60000,3.285871
5,2027-11-15,948,86.777285,2.59726,2.370628
6,2026-11-15,583,95.073194,1.59726,1.457886
7,2025-11-15,218,104.162191,0.59726,0.545144
